In [15]:
import os

In [16]:
%pwd

'd:\\voting_application'

In [17]:
os.chdir("../")

In [18]:
%pwd

'd:\\'

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir : Path
    data_path: Path
    tokenizer_name: Path

In [20]:
from text_summarizer.constants  import *
from text_summarizer.utlis.common import read_yaml,create_directories

ModuleNotFoundError: No module named 'text_summarizer'

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])
    
    def get_data_tranformation_config(self)-> DataTransformationConfig:
        config = self.config.data_tranformation
        create_directories([config.root_dir])
        
        data_tranformation_config= DataTransformationConfig(
            root_dir = config.artifacts_root,
            data_path = config.data_path,
            tokenizer_name =config.tokenizer_name
        )
        return data_tranformation_config

NameError: name 'CONFIG_FILE_PATH' is not defined

In [ ]:
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [ ]:
class DataTransformation:
    def __init__(
            self,config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
    
    
    def convert_examples_to_features(self,example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'], max_lenghth =1024 , truncation = True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'], max_lenghth =1024, truncation = True)

        return{
        'input_ids' : input_encoding['input_ids'],
        'attention_mask' : input_encoding['attention_mask'],
        'labels' : target_encoding['input_ids']}

def converter(self):
    dataset_samsum = load_from_disk(self.config.data_path)
    dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
    dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
    

In [ ]:
try:
    config = ConfigurationManager()
    data_tranformation_config = config.get_data_tranformation_config()
    data_tranformation = DataTransformation(config.get_data_tranformation_config())
    data_tranformation.converter()

except Exception as e:
    raise e